In [1]:
import altair as alt
import pandas as pd

In [118]:
def add_five_mins(group):
    last_row = group.iloc[-1]
    ntime = pd.Timestamp(last_row.Tijd) + pd.Timedelta(5, 'm')
    last_row.Tijd = f"{ntime:%H:%M:%S}"
    return last_row

In [119]:
df = pd.read_excel('data.xlsx', usecols='A:D', nrows=18, dtype='string')
df = pd.concat([
    df,
    df.groupby(['Sessie']).apply(add_five_mins),
], ignore_index=True).sort_values(['Sessie', 'Datum'])
df

,Sessie,Datum,Tijd,Status
0,1,20230113,22:00:00,Inslapen
1,1,20230113,22:33:00,Slapen
2,1,20230114,01:59:00,Wakker
3,1,20230114,02:35:00,Slapen
4,1,20230114,05:27:00,Wakker
5,1,20230114,07:20:00,Slapen
6,1,20230114,07:58:00,Waken
7,1,20230114,08:11:00,Uit bed
18,1,20230114,08:16:00,Uit bed
8,2,20230114,21:39:00,Inslapen


In [120]:
df.Datum = df.Datum.str[:4] + '-' + df.Datum.str[4:6] + '-' + df.Datum.str[6:]

In [135]:
base = (
    alt.Chart(
        df,
        width=600,
        height=80,
    )
    .mark_rect()
    .transform_calculate(
        start = "toDate(datum.Datum + ' ' + datum.Tijd)",
        start_offs = "timeOffset('hours', datum.start, -12)",
    )
    .transform_window(
        groupby = ['Sessie'],
        window = [
            {
                'op': 'lead',
                'field': 'start',
                'as': 'eind',
            },
            {
                'op': 'lead',
                'field': 'start_offs',
                'as': 'eind_offs',
            },
        ]
    )
    .transform_calculate(
        duur = "datum.eind - timeOffset('hours', datum.start, 1)",
    )
)

In [142]:
times = (
    base
    .encode(
        x = (
            alt.X('hoursminutes(start_offs):T')
            .axis(
                title='Tijdstip',
                labelExpr="timeFormat(timeOffset('hours', datum.value, 12), '%H:%M')"
            )
        ),
        x2 = 'hoursminutes(eind_offs):T',
        row = 'Sessie',
        color = (
            alt.Color('Status')
            .scale(
                domain=[
                    "Inslapen",
                    "Slapen",
                    "Wakker",
                    "Waken",
                    "Uit bed"
                ],
                range=[
                    "#4c78a8",
                    "#54a24b",
                    "#e45756",
                    "#eeca3b",
                    "#ff9da6"
                ]
            )
        ),
        tooltip = [
            alt.Tooltip('Sessie:N',),
            alt.Tooltip('Status:N',),
            alt.Tooltip('duur:T', format='%H:%M'),
            alt.Tooltip('start:T', format='%x %H:%M'),
            alt.Tooltip('eind:T', format='%x %H:%M'),
        ]
    )
)
times

alt.Chart(...)